In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [4]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3b
  atmpro = mls
  band = 3b
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 0
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [5, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250)]
  tsfc = 294
  vmax = 720
  vmin = 620
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), (1.66, 1.66)]
  wgt = [(0, 0.6, 0.5, 0.7, 0.8), (0.8, 0.7)]


# Cooling Rate by Layer

CRD             CLIRAD `coolr_bands`           
            layer      coolrg                layer     coolrg
pressure                                                     
0.000312        1  115.318820                    1  74.431084
0.000750        2   49.894432                    2  56.701423
0.001052        3   39.583054                    3  48.889357
0.001476        4   31.229145                    4  39.889818
0.002070        5   24.655570                    5  30.385170
0.002904        6   19.625436                    6  21.716995
0.004074        7   15.801938                    7  15.077387
0.005714        8   12.862952                    8  11.065846
0.008015        9   10.539398                    9   9.084016
0.011243       10    8.617045                   10   8.096043
0.015771       11    6.943739                   11   7.215918
0.022122       12    5.419756                   12   6.078568
0.031031       13    3.950993                   13   4.591206
0.043528       14    2.201139                   14   2.514939
0.061057       15    0.886171                   15   0.757177
0.085645       16    1.381326                   16   0.871334
0.120136       17    2.699170                   17   2.035572
0.168516       18    3.645432                   18   3.057446
0.236378       19    4.581823                   19   4.326237
0.331549       20    5.661577                   20   5.896799
0.465100       21    7.002178                   21   7.564414
0.652400       22    9.057492                   22   9.453330
0.915100       23   11.398736                   23  11.132581
1.283650       24   10.938954                   24  10.285754
1.800600       25    8.435975                   25   8.313426
2.525700       26    6.582664                   26   7.036258
3.542800       27    5.290320                   27   5.644142
4.969550       28    4.305086                   28   4.182044
6.970850       29    3.551954                   29   3.465148
9.778100       30    2.928599                   30   3.277571
13.715850      31    2.308824                   31   2.809744
19.239350      32    1.675905                   32   1.876451
26.987250      33    1.196519                   33   1.153867
37.855300      34    0.876508                   34   0.947213
53.100050      35    0.556945                   35   0.721038
73.887500      36    0.260219                   36   0.307541
97.662500      37    0.127319                   37   0.064804
121.437500     38    0.107399                   38   0.040438
145.212500     39    0.016026                   39  -0.027170
168.987500     40   -0.122929                   40  -0.151721
192.762500     41   -0.113197                   41  -0.124951
216.537500     42    0.019036                   42   0.020269
240.312500     43    0.069182                   43   0.067842
264.087500     44    0.080876                   44   0.077059
287.862500     45    0.083412                   45   0.081353
311.637500     46    0.082093                   46   0.080597
335.412500     47    0.078896                   47   0.079596
359.187500     48    0.074888                   48   0.074622
382.962500     49    0.070858                   49   0.068911
406.737500     50    0.067133                   50   0.063070
430.512500     51    0.063303                   51   0.057240
454.287500     52    0.058694                   52   0.049675
478.062500     53    0.053032                   53   0.042947
501.837500     54    0.046964                   54   0.035454
525.612500     55    0.041368                   55   0.029269
549.387500     56    0.036632                   56   0.024368
573.162500     57    0.032660                   57   0.020601
596.937500     58    0.029314                   58   0.017774
620.712500     59    0.026478                   59   0.015678
644.487500     60    0.024005                   60   0.014068
668.262500     61    0.021880                   61   0.013014
692.037500     62    0.020139                 

# Fluxes by Layer

CRD                       CLIRAD             \
                        flug       fldg      fnetg      i       flug   
pressure    level                                                      
0.000000    1     -17.844520   0.000000 -17.844520     11 -17.741777   
0.000624    2     -17.844949   0.008959 -17.835990     11 -17.742276   
0.000876    3     -17.845156   0.010652 -17.834504     11 -17.742486   
0.001229    4     -17.845416   0.012566 -17.832850     11 -17.742757   
0.001723    5     -17.845739   0.014720 -17.831020     11 -17.743106   
0.002417    6     -17.846144   0.017151 -17.828993     11 -17.743556   
0.003391    7     -17.846655   0.019925 -17.826730     11 -17.744140   
0.004757    8     -17.847307   0.023133 -17.824173     11 -17.744902   
0.006672    9     -17.848149   0.026895 -17.821255     11 -17.745903   
0.009359    10    -17.849255   0.031354 -17.817900     11 -17.747234   
0.013128    11    -17.850732   0.036679 -17.814053     11 -17.749026   
0.018415    12    -17.852752   0.043047 -17.809705     11 -17.751489   
0.025830    13    -17.855596   0.050652 -17.804944     11 -17.754962   
0.036232    14    -17.859787   0.059712 -17.800075     11 -17.760036   
0.050823    15    -17.866817   0.070546 -17.796271     11 -17.768252   
0.071291    16    -17.879098   0.084976 -17.794122     11 -17.781949   
0.100000    17    -17.897425   0.108000 -17.789425     11 -17.801920   
0.140271    18    -17.921451   0.144902 -17.776548     11 -17.827842   
0.196760    19    -17.952330   0.200175 -17.752155     11 -17.860765   
0.275997    20    -17.991162   0.282013 -17.709149     11 -17.901479   
0.387100    21    -18.037835   0.403197 -17.634637     11 -17.949237   
0.543100    22    -18.088160   0.582918 -17.505242     11 -17.999106   
0.761700    23    -18.118981   0.848280 -17.270701     11 -18.028066   
1.068500    24    -18.071791   1.215350 -16.856441     11 -17.980424   
1.498800    25    -17.927692   1.628833 -16.298859     11 -17.837993   
2.102400    26    -17.729855   2.034174 -15.695681     11 -17.641952   
2.949000    27    -17.492941   2.457408 -15.035533     11 -17.407010   
4.136600    28    -17.219391   2.928098 -14.291293     11 -17.132706   
5.802500    29    -16.912701   3.470964 -13.441737     11 -16.820946   
8.139200    30    -16.568722   4.110162 -12.458561     11 -16.463089   
11.417000   31    -16.176254   4.854806 -11.321448     11 -16.050527   
16.014700   32    -15.744560   5.680568 -10.063992     11 -15.593158   
22.464000   33    -15.332105   6.548446  -8.783659     11 -15.149442   
31.510500   34    -14.969333   7.467890  -7.501443     11 -14.751104   
44.200100   35    -14.608211   8.424311  -6.183900     11 -14.350596   
62.000000   36    -14.278153   9.268586  -5.009567     11 -13.981862   
85.775000   37    -14.157337   9.880629  -4.276708     11 -13.830491   
109.550000  38    -14.232659  10.314521  -3.918138     11 -13.884094   
133.325000  39    -14.362687  10.747018  -3.615669     11 -13.979666   
157.100000  40    -14.673555  11.103021  -3.570534     11 -14.253466   
180.875000  41    -15.440575  11.523832  -3.916742     11 -15.021531   
204.650000  42    -16.594456  12.358915  -4.235542     11 -16.214104   
228.425000  43    -17.812162  13.630233  -4.181929     11 -17.474532   
252.200000  44    -19.011415  15.024326  -3.987090     11 -18.709522   
275.975000  45    -20.186743  16.427426  -3.759318     11 -19.915894   
299.750000  46    -21.337717  17.813314  -3.524403     11 -21.093121   
323.525000  47    -22.464248  19.171044  -3.293204     11 -22.243844   
347.300000  48    -23.566017  20.495010  -3.071007     11 -23.365958   
371.075000  49    -24.642206  21.782108  -2.860098     11 -24.460470   
394.850000  50    -25.690906  23.030366  -2.660540     11 -25.525382   
418.625000  51    -26.708953  24.237482  -2.471471     11 -26.557650   
442.400000  52    -27.693710  25.400522  -2.293189     11 -27.554396   
466.175000  53    -28.644922  26.517036  -2.127886     11 -28.517

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')